## Data Ingestion

In [1]:
import pandas as pd

column_names = ['id', 'text', 'label']
twitter_15_train = pd.read_csv("/kaggle/input/twitter15-16/twitter15.train", sep="\t", 
                               header=None, names=column_names)
twitter_15_test = pd.read_csv("/kaggle/input/twitter15-16/twitter15.test", sep="\t", 
                              header=None, names=column_names)
twitter_15_dev = pd.read_csv("/kaggle/input/twitter15-16/twitter15.dev", sep="\t", 
                             header=None, names=column_names)

In [2]:
twitter_15_train.head()

,id,text,label
0,724703995147751424,"american family association gets 500,000 to si...",unverified
1,358591089462099968,this week's top story: george zimmerman wins f...,false
2,775672628493357057,clinton hides failing health? full disclosure ...,unverified
3,364589696573124609,fukushima: highly radioactive water seeping in...,false
4,549927969032916993,a transgender 17-year old left a suicide note ...,unverified


In [3]:
twitter_15_train.shape, twitter_15_test.shape, twitter_15_dev.shape

((1005, 3), (336, 3), (149, 3))

In [4]:
twitter_16_train = pd.read_csv("/kaggle/input/twitter15-16/twitter16.train", sep="\t", 
                               header=None, names=column_names)
twitter_16_test = pd.read_csv("/kaggle/input/twitter15-16/twitter16.test", sep="\t", 
                              header=None, names=column_names)
twitter_16_dev = pd.read_csv("/kaggle/input/twitter15-16/twitter16.dev", sep="\t", 
                             header=None, names=column_names)

In [5]:
twitter_16_train.head()

,id,text,label
0,692929779696275456,ohio lawmakers want to know why state’s epa di...,non-rumor
1,693858804279201794,poor women in india are fighting for the right...,non-rumor
2,693648684857323521,spoiler alert: leo and kate were ridiculously ...,non-rumor
3,620367840902782976,translucent butterfly - beautiful! ' URL,false
4,693939356390653952,michael oher got a text from cam newton during...,non-rumor


In [6]:
twitter_16_train.shape, twitter_16_test.shape, twitter_16_dev.shape

((552, 3), (184, 3), (82, 3))

In [7]:
twitter_train = pd.concat([twitter_15_train, twitter_16_train], ignore_index=True, axis=0)
twitter_test = pd.concat([twitter_15_test, twitter_16_test], ignore_index=True, axis=0)
twitter_dev = pd.concat([twitter_15_dev, twitter_15_test], ignore_index=True, axis=0)

In [8]:
twitter_train.shape, twitter_test.shape, twitter_dev.shape

((1557, 3), (520, 3), (485, 3))

## Data Preparation

In [9]:
LABELS = twitter_train['label'].unique().tolist()
train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3616623573.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = twitter_train['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [10]:
train_data = twitter_train['text'].tolist()

In [11]:
dev_data = twitter_dev['text'].tolist()
dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/3987848631.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = twitter_dev['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [12]:
test_data = twitter_test['text'].tolist()
test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()

/tmp/ipykernel_36/2548705681.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = twitter_test['label'].replace(LABELS, [0, 1, 2, 3]).tolist()


In [13]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [14]:
from transformers import BertTokenizer, BertModel, AdamW

max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [16]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [17]:
num_classes = 4

In [18]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes, dropout=0.1):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [19]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [20]:
model = BERTClassifier(bert_model, num_classes)

In [21]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted')
    }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [23]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [24]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [25]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 49/49 [00:19<00:00,  2.55it/s]



Epoch 1/5
Training Metrics:
Loss: 1.3673
Accuracy: 0.3269
Precision: 0.3310
Recall: 0.3269
F1: 0.2922
Roc_auc: 0.5841

Validation Metrics:
Accuracy: 0.4289
Precision: 0.4172
Recall: 0.4289
F1: 0.3875
Roc_auc: 0.6965


Epoch 2/5: 100%|██████████| 49/49 [00:18<00:00,  2.63it/s]



Epoch 2/5
Training Metrics:
Loss: 1.1804
Accuracy: 0.5138
Precision: 0.5174
Recall: 0.5138
F1: 0.4898
Roc_auc: 0.7707

Validation Metrics:
Accuracy: 0.5526
Precision: 0.5854
Recall: 0.5526
F1: 0.5340
Roc_auc: 0.8016


Epoch 3/5: 100%|██████████| 49/49 [00:18<00:00,  2.63it/s]



Epoch 3/5
Training Metrics:
Loss: 0.8394
Accuracy: 0.7264
Precision: 0.7241
Recall: 0.7264
F1: 0.7214
Roc_auc: 0.9022

Validation Metrics:
Accuracy: 0.6928
Precision: 0.7048
Recall: 0.6928
F1: 0.6923
Roc_auc: 0.8866


Epoch 4/5: 100%|██████████| 49/49 [00:18<00:00,  2.64it/s]



Epoch 4/5
Training Metrics:
Loss: 0.5059
Accuracy: 0.8587
Precision: 0.8582
Recall: 0.8587
F1: 0.8584
Roc_auc: 0.9655

Validation Metrics:
Accuracy: 0.7649
Precision: 0.7672
Recall: 0.7649
F1: 0.7655
Roc_auc: 0.9154


Epoch 5/5: 100%|██████████| 49/49 [00:18<00:00,  2.64it/s]



Epoch 5/5
Training Metrics:
Loss: 0.2668
Accuracy: 0.9351
Precision: 0.9352
Recall: 0.9351
F1: 0.9352
Roc_auc: 0.9904

Validation Metrics:
Accuracy: 0.7794
Precision: 0.7852
Recall: 0.7794
F1: 0.7809
Roc_auc: 0.9263


In [27]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,1.367311,0.326911,0.330967,0.326911,0.292247,0.584145
1,1.180354,0.513809,0.517376,0.513809,0.489821,0.770683
2,0.839388,0.726397,0.724083,0.726397,0.721444,0.902159
3,0.505935,0.858703,0.858224,0.858703,0.858428,0.965467
4,0.266800,0.935132,0.935224,0.935132,0.935168,0.990435


In [28]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.428866,0.417213,0.428866,0.387549,0.696544
1,0.552577,0.585429,0.552577,0.534042,0.801594
2,0.692784,0.704845,0.692784,0.692280,0.886597
3,0.764948,0.767205,0.764948,0.765549,0.915433
4,0.779381,0.785233,0.779381,0.780930,0.926338


In [29]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.7788461538461539,
 'precision': 0.7829947270471463,
 'recall': 0.7788461538461539,
 'f1': 0.7800913398099028,
 'roc_auc': 0.927876154393524}